# Preparation

In [2]:
import pandas as pd
from zipfile import ZipFile
import os
import csv
import janitor
import pandas_gbq
import numpy as np
import re
import io
from google.cloud import bigquery
from google.oauth2 import service_account

# Task 1: The Upload

In the below portion of this notebook the wedge transaction files are read into dataframes, cleaned, and uploaded to Google Big Query. In practice, the "placeholder" in service_file and project ID were my GBQ instance and the proper file. 

In [ ]:
output_dfs = []

service_path = "\\Users\\aidan\\OneDrive\\Desktop\\MSBA_F22\\ADA\\"
service_file = 'placeholder-9bb4f6c64d82.json'  
gbq_proj_id = 'placeholder' 

private_key =service_path + service_file
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

table_name = 'Wedge_2022'

dataset_ = 'Wedge_Wagers'

Now, I read in the uncleaned zip files and establish a list containing the headers.

In [ ]:
zipperDown = os.listdir("\\Users\\aidan\\OneDrive\\Desktop\\MSBA_F22\\ADA\\DBs\\WedgeZipOfZips")
head_insert = """datetime	register_no	emp_no	trans_no	upc	description	trans_type	trans_subtype	trans_status	department	quantity	Scale	cost	unitPrice	total	regPrice	altPrice	tax	taxexempt	foodstamp	wicable	discount	memDiscount	discountable	discounttype	voided	percentDiscount	ItemQtty	volDiscType	volume	VolSpecial	mixMatch	matched	memType	staff	numflag	itemstatus	tenderstatus	charflag	varflag	batchHeaderID	local	organic	display	receipt	card_no	store	branch	match_id	trans_id""".split()

I had an odd issue where my column types were not complying, so I was forced to manually set them. Knowing what I know now, I would have set the datatypes on lists of columns to make this code much more succinct.

In [5]:
for zip in zipperDown:
    with ZipFile("\\Users\\aidan\\OneDrive\\Desktop\\MSBA_F22\\ADA\\DBs\\WedgeZipOfZips\\" + zip, 'r') as zF:
        zip_files = zF.namelist()
      
        for zipped in zip_files:
            currentFile = zF.open(zipped,'r')
            currentFile = io.TextIOWrapper(currentFile, encoding = "utf-8")
        
            sniff = csv.Sniffer().sniff(sample = currentFile.readline())

            for row in currentFile:
                if row[0] == "d" or row[0:2] == "d":
                    df = pd.read_csv(currentFile, sep = sniff.delimiter, encoding = "utf-8")
                    
                else:
                    df = pd.read_csv(currentFile, sep = sniff.delimiter, names = head_insert, encoding = "utf-8")
            df = janitor.clean_names(df)
            
            for column in df:
                df[column] = df[column].replace('\\N',0)

            df['datetime'] = pd.to_datetime(df['datetime'],format='%Y-%m-%d %H:%M:%S')
            df['altprice'] = df['altprice'].astype(str)
            df['local'] = df['local'].fillna(0)
            df['local'] = df['local'].replace('\\N',0)
            df['local'] = df['local'].astype(int)
            df['register_no'] = df['register_no'].astype(int)
            df['emp_no'] = df['emp_no'].astype(int)
            df['trans_no'] = df['trans_no'].astype(int)
            df['upc'] = df['upc'].astype(str)
            df['description'] = df['description'].astype(str)
            df['trans_type'] = df['trans_type'].astype(str)
            df['trans_subtype'] = df['trans_subtype'].astype(str)
            df['trans_status'] = df['trans_status'].astype(str)
            df['department'] = df['department'].astype(int)
            df['quantity'] = df['quantity'].astype(float)
            df['scale'] = df['scale'].astype(int)
            df['cost'] = df['cost'].astype(float)
            df['unitprice'] = df['unitprice'].astype(float)
            df['total'] = df['total'].astype(float)
            df['regprice'] = df['regprice'].astype(str)
            
            df['tax'] = df['tax'].astype(float)
            df['taxexempt'] = df['taxexempt'].astype(float)
            df['foodstamp'] = df['foodstamp'].astype(float)
            df['wicable'] = df['wicable'].astype(float)
            df['discount'] = df['discount'].astype(float)
            df['memdiscount'] = df['memdiscount'].astype(float)
            df['discountable'] = df['discountable'].astype(float)
            df['discounttype'] = df['discounttype'].astype(float)
            df['voided'] = df['voided'].astype(float)
            df['percentdiscount'] = df['percentdiscount'].astype(float)
            df['itemqtty'] = df['itemqtty'].astype(float)
            df['voldiscype'] = df['voldisctype'].astype(float)
            df['volume'] = df['volume'].astype(float)
            df['volspecial'] = df['volspecial'].astype(float)
            df['mixmatch'] = df['mixmatch'].astype(float)
            df['matched'] = df['matched'].astype(float)
            df['memtype'] = df['memtype'].astype(bool)
            df['staff'] = df['staff'].astype(bool)
            df['numflag'] = df['numflag'].astype(float)
            df['itemstatus'] = df['itemstatus'].astype(str)
            df['tenderstatus'] = df['tenderstatus'].astype(float)
            df['charflag'] = df['charflag'].astype(str)
            df['varflag'] = df['varflag'].astype(float)
            df['batchheaderid'] = df['batchheaderid'].astype(bool)
            df['organic'] = df['organic'].astype(str)
            df['display'] = df['display'].astype(bool)
            df['receipt'] = df['receipt'].astype(float)
            df['card_no'] = df['card_no'].astype(int)
            df['store'] = df['store'].astype(float)
            df['branch'] = df['branch'].astype(float)
            df['match_id'] = df['match_id'].astype(float)
            df['trans_id'] = df['trans_id'].astype(float)
            
            for column in df:
                
                if df[column].dtypes == object:
                    df = df.astype({column: 'str'})
                if df[column].dtypes == str:
                    df[column] = df[column].replace(np.nan,'',regex=True)
                    df[column] = df[column].replace('nan','',regex=True)
                
                if df[column].dtypes == int:
                    df[column] = df[column].fillna(0)
                    df[column] = df[column].replace('\\N',0)
                    df[column] = df[column].astype(int)
                if df[column].dtypes == float:
                    df[column] = df[column].fillna(0)
                    df[column] = df[column].replace('\\N',0)
                    df[column] = df[column].astype(float)
            
            tempTuple = os.path.splitext(zipped)

            filename = tempTuple[0]
            
            table_id = ".".join([gbq_proj_id,dataset_,filename])
            client.delete_table(table_id, not_found_ok=True)
            pandas_gbq.to_gbq(df, table_id, project_id= 'bigquerywagers',if_exists="replace")

bigquerywagers:Wedge_B_level
bigquerywagers:Wedge_Wagers
bigquerywagers:Wedge_clean
bigquerywagers:dram_shop
bigquerywagers:umt_msba_aw
Completed Cleaning. Please god work.


100%|██████████| 1/1 [00:00<?, ?it/s]


Completed Cleaning. Please god work.


100%|██████████| 1/1 [00:00<?, ?it/s]


Completed Cleaning. Please god work.


...

100%|██████████| 1/1 [00:00<?, ?it/s]
C:\Users\aidan\AppData\Local\Temp\ipykernel_6780\1994732405.py:34: DtypeWarning: Columns (18,36,37,41,43,44,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(currentFile, sep = sniff.delimiter, names = head_insert, encoding = "utf-8")


Completed Cleaning. Please god work.


100%|██████████| 1/1 [00:00<00:00, 1003.66it/s]
C:\Users\aidan\AppData\Local\Temp\ipykernel_6780\1994732405.py:34: DtypeWarning: Columns (18,36,37,41,43,44,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(currentFile, sep = sniff.delimiter, names = head_insert, encoding = "utf-8")


Completed Cleaning. Please god work.


100%|██████████| 1/1 [00:00<?, ?it/s]
